# Forecasting con diferentes modelos

In [1]:
from __future__ import annotations

from typing import Tuple

import pandas as pd

# Lectura de datos

In [2]:
sales = pd.read_csv("sales.csv")

In [3]:
sales.index = pd.to_datetime(sales["Week"], unit="W")
sales.index.name = "Week Date"

In [4]:
sales

,Week,Product,Sales,Promotion,Holiday
Week Date,,,,,
1970-01-08,1,Product_1,152.0,1,0
1970-01-15,2,Product_1,485.0,0,0
1970-01-22,3,Product_1,398.0,0,0
1970-01-29,4,Product_1,320.0,0,0
1970-02-05,5,Product_1,156.0,0,0
1970-02-12,6,Product_1,121.0,1,0
1970-02-19,7,Product_1,238.0,0,0
1970-02-26,8,Product_1,70.0,1,0
1970-03-05,9,Product_1,152.0,1,0


In [5]:
sales.shape

(30, 5)

# SARIMAX

## División de datos

In [6]:
def split_sales_sarimax(
    sales: pd.DataFrame,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    sales_transformed = sales.copy()

    weeks_to_forecast = sales_transformed[sales_transformed["Week"] > 28]
    train = sales_transformed[sales_transformed["Week"] <= 22]
    test = sales_transformed[
        (sales_transformed["Week"] > 22) & (sales_transformed["Week"] <= 28)
    ]

    train = train.drop(columns=["Week", "Product"])
    test = test.drop(columns=["Week", "Product"])
    weeks_to_forecast = weeks_to_forecast.drop(columns=["Week", "Product"])

    return train, test, weeks_to_forecast

In [7]:
train_sarimax, test_sarimax, weeks_to_forecast_sarimax = split_sales_sarimax(sales)

In [8]:
display(train_sarimax)
display(test_sarimax)
display(weeks_to_forecast_sarimax)

,Sales,Promotion,Holiday
Week Date,,,
1970-01-08,152.0,1,0
1970-01-15,485.0,0,0
1970-01-22,398.0,0,0
1970-01-29,320.0,0,0
1970-02-05,156.0,0,0
1970-02-12,121.0,1,0
1970-02-19,238.0,0,0
1970-02-26,70.0,1,0
1970-03-05,152.0,1,0


,Sales,Promotion,Holiday
Week Date,,,
1970-06-11,463.0,0,0
1970-06-18,343.0,0,0
1970-06-25,435.0,1,0
1970-07-02,241.0,0,0
1970-07-09,493.0,1,1
1970-07-16,326.0,0,0


,Sales,Promotion,Holiday
Week Date,,,
1970-07-23,NaN,1,0
1970-07-30,NaN,0,1


## Modelo SARIMAX

In [9]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [10]:
sarimax_model = SARIMAX(
    train_sarimax["Sales"],
    exog=train_sarimax[["Promotion", "Holiday"]],
    order=(2, 1, 2),
    seasonal_order=(0, 0, 0, 4),
)
sarimax_results = sarimax_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.25120D+00    |proj g|=  5.86290D-01

At iterate    5    f=  5.87308D+00    |proj g|=  4.12790D-02

At iterate   10    f=  5.86427D+00    |proj g|=  4.79474D-03

At iterate   15    f=  5.85552D+00    |proj g|=  5.13710D-04

At iterate   20    f=  5.85265D+00    |proj g|=  3.46012D-03

At iterate   25    f=  5.85006D+00    |proj g|=  6.95314D-05

At iterate   30    f=  5.84998D+00    |proj g|=  8.32196D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

/home/carroyo/other_files/mercadolibre-test/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/carroyo/other_files/mercadolibre-test/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/carroyo/other_files/mercadolibre-test/.venv/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
 This problem is unconstrained.


In [11]:
sarimax_forecast = sarimax_results.forecast(
    steps=len(test_sarimax), exog=test_sarimax[["Promotion", "Holiday"]]
)

In [12]:
sarimax_forecast

1970-06-11    335.709003
1970-06-18    319.081696
1970-06-25    210.843323
1970-07-02    312.994459
1970-07-09     71.328840
1970-07-16    312.493202
Freq: W-THU, Name: predicted_mean, dtype: float64

# División de datos para los demás modelos

Se usan lagged features y rolling windows. No se pueden lags o ventanas muy grandes porque el dataset es muy pequeño.

In [15]:
def split_sales_models(
    sales: pd.DataFrame,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    sales_transformed = sales.copy()

    sales_transformed["Sales Lag_1"] = sales_transformed["Sales"].shift(1)
    sales_transformed["Sales Lag_2"] = sales_transformed["Sales"].shift(2)
    sales_transformed["Sales Lag_4"] = sales_transformed["Sales"].shift(4)
    sales_transformed["Rolling_Mean_3"] = (
        sales_transformed["Sales"].rolling(window=3).mean()
    )
    sales_transformed["Rolling_Std_3"] = (
        sales_transformed["Sales"].rolling(window=3).std()
    )

    weeks_to_forecast = sales_transformed[sales_transformed["Week"] > 28]
    train = sales_transformed[sales_transformed["Week"] <= 22]
    test = sales_transformed[
        (sales_transformed["Week"] > 22) & (sales_transformed["Week"] <= 28)
    ]

    # Keep "Week" to capture any linear trend
    train = train.drop(columns=["Product"]).dropna()
    test = test.drop(columns=["Product"])
    weeks_to_forecast = weeks_to_forecast.drop(columns=["Product"])

    return train, test, weeks_to_forecast

In [16]:
train_models, test_models, weeks_to_forecast_models = split_sales_models(sales)

In [17]:
display(train_models)
display(test_models)
display(weeks_to_forecast_models)

,Week,Sales,Promotion,Holiday,Sales Lag_1,Sales Lag_2,Sales Lag_4,Rolling_Mean_3,Rolling_Std_3
Week Date,,,,,,,,,
1970-02-05,5,156.0,0,0,320.0,398.0,152.0,291.333333,123.520579
1970-02-12,6,121.0,1,0,156.0,320.0,485.0,199.000000,106.240294
1970-02-19,7,238.0,0,0,121.0,156.0,398.0,171.666667,60.052755
1970-02-26,8,70.0,1,0,238.0,121.0,320.0,143.000000,86.133617
1970-03-05,9,152.0,1,0,70.0,238.0,156.0,153.333333,84.007936
1970-03-12,10,171.0,0,0,152.0,70.0,121.0,131.000000,53.674948
1970-03-19,11,264.0,0,0,171.0,152.0,238.0,195.666667,59.936077
1970-03-26,12,380.0,1,0,264.0,171.0,70.0,271.666667,104.710713
1970-04-02,13,137.0,1,1,380.0,264.0,152.0,260.333333,121.541488


,Week,Sales,Promotion,Holiday,Sales Lag_1,Sales Lag_2,Sales Lag_4,Rolling_Mean_3,Rolling_Std_3
Week Date,,,,,,,,,
1970-06-11,23,463.0,0,0,393.0,307.0,199.0,387.666667,78.136632
1970-06-18,24,343.0,0,0,463.0,393.0,358.0,399.666667,60.277138
1970-06-25,25,435.0,1,0,343.0,463.0,307.0,413.666667,62.780039
1970-07-02,26,241.0,0,0,435.0,343.0,393.0,339.666667,97.042946
1970-07-09,27,493.0,1,1,241.0,435.0,463.0,389.666667,131.974745
1970-07-16,28,326.0,0,0,493.0,241.0,343.0,353.333333,128.204264


,Week,Sales,Promotion,Holiday,Sales Lag_1,Sales Lag_2,Sales Lag_4,Rolling_Mean_3,Rolling_Std_3
Week Date,,,,,,,,,
1970-07-23,29,NaN,1,0,326.0,493.0,435.0,NaN,NaN
1970-07-30,30,NaN,0,1,NaN,326.0,241.0,NaN,NaN


In [18]:
X_train = train_models.drop(columns=["Week", "Sales"])
y_train = train_models["Sales"]
X_test = test_models.drop(columns=["Week", "Sales"])
y_test = test_models["Sales"]

# Random Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
rf = RandomForestRegressor(
    n_estimators=100, max_features=4, max_depth=10, random_state=42
)

In [21]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, max_features=4, random_state=42)

## Solo se puede predecir iterativamente

In [22]:
test_iterative = X_test.copy()
date_ranges = X_train.iloc[-4:].index

test_iterative = pd.concat([X_train.iloc[-4:], test_iterative])
test_iterative.loc[date_ranges, "Predicted Sales"] = y_train
test_iterative.sort_index(inplace=True)

# iteration_dates are the dates in `test_iterative` not in `date_ranges`
iteration_dates = test_iterative.index.difference(date_ranges)

In [23]:
for i, dt in enumerate(iteration_dates):
    # if i > 0:
    #     break
    predicted_sales = rf.predict(
        test_iterative.drop(columns=["Predicted Sales"]).loc[[dt]]
    )[0]
    test_iterative.loc[dt, "Predicted Sales"] = predicted_sales

    if i == len(iteration_dates) - 1:
        break

    next_dt = dt + pd.DateOffset(weeks=1)

    # Update the lag features
    test_iterative.loc[next_dt, "Sales Lag_1"] = test_iterative.loc[
        dt, "Predicted Sales"
    ]
    test_iterative.loc[next_dt, "Sales Lag_2"] = test_iterative.loc[
        dt - pd.DateOffset(weeks=1), "Predicted Sales"
    ]
    test_iterative.loc[next_dt, "Sales Lag_4"] = test_iterative.loc[
        dt - pd.DateOffset(weeks=3), "Predicted Sales"
    ]

    # Update the rolling features
    test_iterative.loc[next_dt, "Rolling_Mean_3"] = test_iterative.loc[
        dt - pd.DateOffset(weeks=1) : dt, "Predicted Sales"
    ].mean()
    test_iterative.loc[next_dt, "Rolling_Std_3"] = test_iterative.loc[
        dt - pd.DateOffset(weeks=1) : dt, "Predicted Sales"
    ].std()

In [24]:
test_iterative = test_iterative[test_iterative.index.isin(iteration_dates)]

In [25]:
test_iterative

,Promotion,Holiday,Sales Lag_1,Sales Lag_2,Sales Lag_4,Rolling_Mean_3,Rolling_Std_3,Predicted Sales
Week Date,,,,,,,,
1970-06-11,0,0,393.00,307.00,199.00,387.666667,78.136632,326.49
1970-06-18,0,0,326.49,393.00,358.00,359.745000,47.029672,326.96
1970-06-25,1,0,326.96,326.49,307.00,326.725000,0.332340,298.75
1970-07-02,0,0,298.75,326.96,393.00,312.855000,19.947482,349.01
1970-07-09,1,1,349.01,298.75,326.49,323.880000,35.539187,292.11
1970-07-16,0,0,292.11,349.01,326.96,320.560000,40.234376,354.43


# Prophet
Modelo pre-entrenado de Facebook.

In [26]:
from prophet import Prophet

/home/carroyo/other_files/mercadolibre-test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_prophet = train_sarimax.copy()
train_prophet["Date"] = train_prophet.index
train_prophet = train_prophet[["Date", "Sales"]].rename(
    columns={"Date": "ds", "Sales": "y"}
)

In [43]:
model_pro = Prophet()
model_pro.fit(train_prophet)

22:58:58 - cmdstanpy - INFO - Chain [1] start processing


22:58:58 - cmdstanpy - INFO - Chain [1] done processing


In [47]:
# Make future predictions
future = model_pro.make_future_dataframe(periods=6, freq="W", include_history=False)
forecast = model_pro.predict(future)

In [49]:
forecast["ds"] = test_sarimax.index

In [51]:
prophet_forecast = (
    forecast[["ds", "yhat"]].set_index("ds").rename(columns={"yhat": "Sales"})
)
prophet_forecast.index.name = "Week Date"

In [52]:
prophet_forecast

,Sales
Week Date,
1970-06-11,281.217827
1970-06-18,283.400780
1970-06-25,285.583733
1970-07-02,287.766685
1970-07-09,289.949638
1970-07-16,292.132591


# Comparar los modelos

Se usa el `mean_absolute_error` para comparar los modelos.

In [53]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# Get the error for all the models
sarimax_error = mean_absolute_error(test_sarimax["Sales"], sarimax_forecast)
models_error = mean_absolute_error(
    test_models["Sales"], test_iterative["Predicted Sales"]
)
prophet_error = mean_absolute_error(test_sarimax["Sales"], prophet_forecast["Sales"])

In [60]:
errors = {
    "SARIMAX": sarimax_error,
    "Prophet": prophet_error,
    "Random Forest": models_error,
}

sorted_errors = dict(sorted(errors.items(), key=lambda x: x[1]))

# Print the sorted errors and format the output to 2 decimal places
print("Errores organizados de menor a mayor:")
for model, error in sorted_errors.items():
    print(f"{model}: {error:.2f}")

Errores organizados de menor a mayor:
Random Forest: 104.36
Prophet: 112.41
SARIMAX: 147.09


# Predicción usando el mejor modelo: RandomForestRegressor

Con `RandomForestRegressor` solo se puede predecir iterativamente. Sin embargo, las predicciones caen muy bien dentro del orden de magnitud.

Fiteamos ahora sí con el dataset completo hasta la semana 28.

In [105]:
X_final = pd.concat([X_train, X_test])
y_final = pd.concat([y_train, y_test])

rf_best = RandomForestRegressor(
    n_estimators=100, max_features=4, max_depth=10, random_state=42
)
rf_best.fit(X_final, y_final)

RandomForestRegressor(max_depth=10, max_features=4, random_state=42)

In [106]:
predict_iterative = weeks_to_forecast_models.drop(columns=["Sales", "Week"]).copy()
date_ranges = X_final.iloc[-4:].index

predict_iterative = pd.concat([X_final.iloc[-4:], predict_iterative])
predict_iterative.loc[date_ranges, "Predicted Sales"] = y_test
predict_iterative.sort_index(inplace=True)

# iteration_dates are the dates in `predict_iterative` not in `date_ranges`
iteration_dates = predict_iterative.index.difference(date_ranges)

In [107]:
for i, dt in enumerate(iteration_dates):
    # Update the lag features
    predict_iterative.loc[dt, "Sales Lag_1"] = predict_iterative.loc[
        dt - pd.DateOffset(weeks=1), "Predicted Sales"
    ]
    predict_iterative.loc[dt, "Sales Lag_2"] = predict_iterative.loc[
        dt - pd.DateOffset(weeks=2), "Predicted Sales"
    ]
    predict_iterative.loc[dt, "Sales Lag_4"] = predict_iterative.loc[
        dt - pd.DateOffset(weeks=4), "Predicted Sales"
    ]

    # Update the rolling features
    predict_iterative.loc[dt, "Rolling_Mean_3"] = predict_iterative.loc[
        dt - pd.DateOffset(weeks=3) : dt - pd.DateOffset(weeks=1), "Predicted Sales"
    ].mean()
    predict_iterative.loc[dt, "Rolling_Std_3"] = predict_iterative.loc[
        dt - pd.DateOffset(weeks=3) : dt - pd.DateOffset(weeks=1), "Predicted Sales"
    ].std()

    predicted_sales = rf_best.predict(
        predict_iterative.drop(columns=["Predicted Sales"]).loc[[dt]]
    )[0]
    predict_iterative.loc[dt, "Predicted Sales"] = predicted_sales

In [108]:
predict_iterative = predict_iterative[predict_iterative.index.isin(iteration_dates)]

# Predicciones finales

In [109]:
predict_iterative

,Promotion,Holiday,Sales Lag_1,Sales Lag_2,Sales Lag_4,Rolling_Mean_3,Rolling_Std_3,Predicted Sales
Week Date,,,,,,,,
1970-07-23,1,0,326.00,493.0,435.0,353.333333,128.204264,383.47
1970-07-30,0,1,383.47,326.0,241.0,400.823333,84.841639,398.37
